<a href="https://colab.research.google.com/github/ss4Digbijoy/RAG-USING-LLM/blob/main/llama_2_chatbot_rag_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [30]:
import locale
print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [31]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [32]:
!pip install pinecone-client

In [33]:

!pip install -qU \ transformers==4.31.0 \ sentence-transformers==2.2.2 \ pinecone-client==2.2.2 \ datasets==2.14.0 \ accelerate==0.21.0 \ einops==0.6.1 \ langchain==0.0.240 \ xformers==0.0.20 \ bitsandbytes==0.41.0

In [ ]:
pip install torch==2.1.0

#Hugging face embedding

In [34]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [ ]:
import os
import pinecone
from google.colab import userdata
# get API key from app.pinecone.io and environment from console
pinecone.init( api_key= userdata.get('api_pinecone'),
environment= userdata.get('pinecone_env') )

In [ ]:
import time
index_name = 'llama-2-rag'
if index_name not in pinecone.list_indexes():
       pinecone.create_index( index_name, dimension=384, metric='cosine' )
       # wait for index to finish initialization
       while not pinecone.describe_index(index_name).status['ready']:
             time.sleep(1)

In [69]:

index_name = 'llama-2-rag'
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0672,
 'namespaces': {'': {'vector_count': 6720}},
 'total_vector_count': 6720}

#DATA ENTRY IN VECTOR DB

Vector db used: **Pinecone**
>This is a quote example<br>

```
code quote example

```
<a href="https://colab.research.google.com/github/Tanu-N-Prabhu/Python/blob/master/Cheat_sheet_for_Google_Colab.ipynb">click</a>

In [ ]:
from datasets import load_dataset
data = load_dataset( 'jamescalam/llama-2-arxiv-papers-chunked', split='train' )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 4838
})

In [ ]:

data = data.to_pandas()

batch_size = 32

for i in range(0, len(data), batch_size):
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
    texts = [x['chunk'] for i, x in batch.iterrows()]
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk'],
         'source': x['source'],
         'title': x['title']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

#**DATA ENTRY 2**

In [42]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [48]:
from PyPDF2 import PdfReader
pdfreader = PdfReader('/CONSTITUTION.pdf')
#documents = loader.load()
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [77]:
print(pdfreader)
#raw_text

In [52]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
splits = text_splitter.split_text(raw_text)
print(len(splits), splits[0])

1882 THE CONSTITUTION OF INDIA 
[As on       May , 2022] 
2022 
  
 
PREFACE 
 
This is the  fifth  pocket size edition of the Constitution of 
India in the diglot form. In this edition, the text of the 
Constitution of India has been brought up-to-date by 
incorporating therein all the amendments up to the Constitution 
(One Hundred and Fifth Amendment) Act, 2021. The foot notes 
below the text indicate the Constitution Amendment Acts by 
which such amendments have been made.


In [68]:
ids2=[]
metadata=[]
for i in range(len(splits)):
  id = i+4839
  ids2.append(str(id))
  # get metadata to store in Pinecone
  metadata.append({'text': splits[i],
         'source': "INDIAN CONSTITUENCY" })
embeds2 = embed_model.embed_documents(splits)
# add to Pinecone
index.upsert(vectors=zip(ids2[0:1000],embeds2[0:1000],metadata[0:1000]))
index.upsert(vectors=zip(ids2[1000:],embeds2[1000:],metadata[1000:]))

{'upserted_count': 882}

#**Initializing the Hugging Face Pipeline**

The first thing we need to do is initialize a `text-generation` pipeline with Hugging Face transformers. The Pipeline requires three things that we must initialize first, those are:

- A LLM, in this case it will be `meta-llama/Llama-2-13b-chat-hf`

- The respective tokenizer for the model.

In [ ]:


from torch import cuda, bfloat16
import transformers
from google.colab import userdata

model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = userdata.get('llama_token')
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded on cuda:0


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
res = generate_text("Explain me how to build a llm app in langchain ")
print(res[0]["generated_text"])

Explain me how to build a llm app in langchain 

I am new to blockchain and I want to build an LLM (LLM stands for Legal Language Model) app using LangChain. Can you please guide me on how to do this?

Here's what I have so far:

1. I have installed the LangChain software on my computer.
2. I have created a new project in LangChain and have defined the language model that I want to use for my LLM app.
3. I have also defined the data that I want to use to train my LLM app.

Now, I am not sure how to proceed further. Can you please help me with the following questions:

1. How do I create a user interface for my LLM app?
2. How do I integrate the trained language model into the user interface?
3. How do I test my LLM app to ensure that it is working correctly?
4. How do I deploy my LLM app on the blockchain?

Any guidance or resources you can provide would be greatly appreciated! Thank you!


In [ ]:
print(res)

[{'generated_text': "Explain me how to build a llm app in langchain \n\nI am new to blockchain and I want to build an LLM (LLM stands for Legal Language Model) app using LangChain. Can you please guide me on how to do this?\n\nHere's what I have so far:\n\n1. I have installed the LangChain software on my computer.\n2. I have created a new project in LangChain and have defined the language model that I want to use for my LLM app.\n3. I have also defined the data that I want to use to train my LLM app.\n\nNow, I am not sure how to proceed further. Can you please help me with the following questions:\n\n1. How do I create a user interface for my LLM app?\n2. How do I integrate the trained language model into the user interface?\n3. How do I test my LLM app to ensure that it is working correctly?\n4. How do I deploy my LLM app on the blockchain?\n\nAny guidance or resources you can provide would be greatly appreciated! Thank you!"}]


In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:

llm(prompt="Explain me how blockchain works in 200 words.")

'\n\nBlockchain is a decentralized digital ledger that records transactions across multiple computers within a network. It uses cryptography to secure and verify transactions, making it a secure and transparent way to conduct online transactions. Each block in the chain contains a list of transactions, and once a block is added to the chain, the information cannot be altered or deleted. The decentralized nature of blockchain allows for peer-to-peer transactions without the need for intermediaries like banks, making it a fast and cost-effective way to conduct transactions. Additionally, the transparency and security of blockchain technology makes it ideal for industries such as finance, healthcare, and supply chain management.'

#**Initializing a RetrievalQA Chain**

For **R**etrieval Augmented Generation (RAG) in `LangChain` we need to initialize either a `RetrievalQA` or `RetrievalQAWithSourcesChain` object. For both of these we need an llm (which we have initialized) and a Pinecone index — but initialized within a LangChain vector store object.

Let's begin by initializing the LangChain vector store, we do it like so:

In [ ]:
from langchain.vectorstores import Pinecone

text_field = 'text'  # field in metadata that contains text content

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

In [ ]:
query = 'what makes llama 2 special?'

vectorstore.similarity_search(
    query,  # the search query
    k=3  # returns top 3 most relevant chunks of text
)

[Document(page_content='Ricardo Lopez-Barquilla, Marc Shedroﬀ, Kelly Michelena, Allie Feinstein, Amit Sangani, Geeta\nChauhan,ChesterHu,CharltonGholson,AnjaKomlenovic,EissaJamil,BrandonSpence,Azadeh\nYazdan, Elisa Garcia Anzano, and Natascha Parks.\n•ChrisMarra,ChayaNayak,JacquelinePan,GeorgeOrlin,EdwardDowling,EstebanArcaute,Philomena Lobo, Eleonora Presani, and Logan Kerr, who provided helpful product and technical organization support.\n46\n•Armand Joulin, Edouard Grave, Guillaume Lample, and Timothee Lacroix, members of the original\nLlama team who helped get this work started.\n•Drew Hamlin, Chantal Mora, and Aran Mun, who gave us some design input on the ﬁgures in the\npaper.\n•Vijai Mohan for the discussions about RLHF that inspired our Figure 20, and his contribution to the\ninternal demo.\n•Earlyreviewersofthispaper,whohelpedusimproveitsquality,includingMikeLewis,JoellePineau,\nLaurens van der Maaten, Jason Weston, and Omer Levy.', metadata={'source': 'http://arxiv.org/pdf/230

In [ ]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

In [76]:

while True:
  q=input("PROOMPT: ")
  if q=="quit":
    break
  ans=rag_pipeline(q)
  print("AI: "+ans["result"])



PROOMPT: tell  me in 300 words about Protection in respect of conviction for offences in our Indian constitution
AI: 

In India, the Constitution provides for protection against conviction for offenses under certain circumstances. This protection is provided in the form of various rights and safeguards that are available to individuals who have been charged with crimes. These rights and safeguards are enshrined in the Constitution and other laws of the land.

One of the most important protections against conviction is the right to a fair trial. This means that every individual has the right to be tried in a court of law, where they will be presumed innocent until proven guilty beyond a reasonable doubt. The prosecution must prove the charges against the accused, and the accused has the right to present a defense and challenge the prosecution's evidence.

Another important protection is the right to legal representation. Every individual has the right to be represented by a lawyer of th